In [1]:
import os

In [2]:
pwd

'c:\\Users\\Cynthia\\Documents\\Learning Data Science\\projects\\chicken-disease\\chicken-disease\\research'

In [3]:
os.chdir('../')

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    parmas_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [10]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

class configurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self):
        config=self.config.data_iingestion

        create_directories([config.root_dir])

        prepare_base_model_config=PrepareBaseModelConfig(
                    root_dir= Path(config.root_dir),
                    base_model_path= Path(config.base_model_path),
                    updated_base_model_path= Path(config.updated_base_model),
                    parmas_image_size = self.params.image_size,
                    params_learning_rate = self.params.learning_rate,
                    params_include_top = self.params.include_top,
                    params_weights =self.params.weights,
                    params_classes = self.params.classes
        )
        return prepare_base_model_config

In [4]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config=config

    def get_base_model(self):
        self.model=tf.keras.application.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_image=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable=False
        elif (freeze_till is not None) and (freeze_till>0):
            for layer in model.layers[:-freeze_till]:
                model.trainable=False

        flatten_in=tf.keras.layers.Flatten()(model.outpt)
        prediction=tf.keras.layers.Dense(units=classes, activation="softmax" )(flatten_in)

        full_model=tf.keras.models.model(inputs=model.inputs, output=prediction)

        full_model.compile(loss=tf.keras.losses.catergoricalcrossentropy(),
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            metrics=["accuracy"])
        
        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model=self._get_base_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model:tf.keras.Model):
        model.save(path)



ImportError: DLL load failed while importing _pywrap_tf2: A dynamic link library (DLL) initialization routine failed.